# MCP Toolbox


## Prerequisites

1. Follow the setup instructions https://github.com/vellum-ai/vellum-python-sdks/tree/c7e2ba1fd0bed4c78cfeafc22c98551b20f9044d/examples/workflows/mcp_toolbox


In [1]:
from vellum.workflows.inputs import BaseInputs


class Inputs(BaseInputs):
    query: str

Define an Agent Node that has a mcp tool pointing to mcp server setup by `toolbox`


In [ ]:
from vellum.client.types.chat_message_prompt_block import ChatMessagePromptBlock
from vellum.client.types.plain_text_prompt_block import PlainTextPromptBlock
from vellum.client.types.rich_text_prompt_block import RichTextPromptBlock
from vellum.client.types.variable_prompt_block import VariablePromptBlock
from vellum.workflows.nodes.displayable.tool_calling_node import ToolCallingNode
from vellum.workflows.types.definition import MCPServer

class Agent(ToolCallingNode):
    ml_model = "gpt-4o-mini"
    blocks = [
        ChatMessagePromptBlock(
            chat_role="SYSTEM",
            blocks=[
                RichTextPromptBlock(
                    blocks=[
                        PlainTextPromptBlock(
                            text="""
You're a helpful hotel assistant. You handle hotel searching, booking and
cancellations. When the user searches for a hotel, mention it's name, id,
location and price tier. Always mention hotel ids while performing any
searches. This is very important for any operations. For any bookings or
cancellations, please provide the appropriate confirmation. Be sure to
update checkin or checkout dates if mentioned by the user.
Don't ask for confirmations from the user.
"""
                        ),
                    ],
                ),
            ],
        ),
        ChatMessagePromptBlock(
            chat_role="USER",
            blocks=[
                VariablePromptBlock(
                    input_variable="query",
                ),
            ],
        ),
    ]
    functions = [
        MCPServer(
            name="toolbox",
            url="http://127.0.0.1:5000/mcp",
        ),
    ]
    prompt_inputs = {
        "query": Inputs.query,
    }


In [3]:
from vellum.workflows import BaseWorkflow
from vellum.workflows.state.base import BaseState


class MCPToolboxWorkflow(BaseWorkflow[Inputs, BaseState]):
    graph = Agent

    class Outputs:
        text = Agent.Outputs.text
        chat_history = Agent.Outputs.chat_history

Interactive chat with mcp toolbox

`Find hotels in Basel with Basel in its name.`


In [4]:
workflow = MCPToolboxWorkflow()

query = input("Hi! I'm an your hotel booking assistant. What can I do for you today? ")
inputs = Inputs(query=query)

print("Running workflow with inputs: ", query)

terminal_event = workflow.run(inputs=inputs)
if terminal_event.name == "workflow.execution.fulfilled":
    print("Answer:", terminal_event.outputs["text"])
elif terminal_event.name == "workflow.execution.rejected":
    print("Workflow rejected", terminal_event.error.code, terminal_event.error.message)


Running workflow with inputs:  Find hotels in Basel with Basel in its name.
Answer: Following are the hotels in Basel with Basel in their name:
- Hilton Basel, with an id of 1, is a Luxury hotel in Basel.
- Hyatt Regency Basel, with an id of 3, is a Upper Upscale hotel in Basel.
- Holiday Inn Basel, with an id of 8, is a Upper Midscale hotel in Basel.
